In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split

from torch_explain.models.explainer import MuExplainer
from experiments.data.load_datasets import load_mimic

In [2]:
x, y, concept_names = load_mimic()

dataset = TensorDataset(x, y)

train_size = int(len(dataset) * 0.5)
val_size = (len(dataset) - train_size) // 2
test_size = len(dataset) - train_size - val_size
train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_data, batch_size=train_size)
val_loader = DataLoader(val_data, batch_size=val_size)
test_loader = DataLoader(test_data, batch_size=test_size)

n_concepts = next(iter(train_loader))[0].shape[1]
n_classes = 2

print(concept_names)
print(n_concepts)
print(n_classes)

['aline_flg', 'gender_num', 'sepsis_flg', 'chf_flg', 'afib_flg', 'renal_flg', 'liver_flg', 'copd_flg', 'cad_flg', 'stroke_flg', 'mal_flg', 'resp_flg', 'abg_count_LOW', 'abg_count_NORMAL', 'abg_count_HIGH', 'age_LOW', 'age_NORMAL', 'age_HIGH', 'bmi_LOW', 'bmi_NORMAL', 'bmi_HIGH', 'bun_first_LOW', 'bun_first_NORMAL', 'bun_first_HIGH', 'chloride_first_LOW', 'chloride_first_NORMAL', 'chloride_first_HIGH', 'creatinine_first_LOW', 'creatinine_first_NORMAL', 'creatinine_first_HIGH', 'day_icu_intime_num_LOW', 'day_icu_intime_num_NORMAL', 'day_icu_intime_num_HIGH', 'hgb_first_LOW', 'hgb_first_NORMAL', 'hgb_first_HIGH', 'hospital_los_day_LOW', 'hospital_los_day_NORMAL', 'hospital_los_day_HIGH', 'hour_icu_intime_LOW', 'hour_icu_intime_NORMAL', 'hour_icu_intime_HIGH', 'hr_1st_LOW', 'hr_1st_NORMAL', 'hr_1st_HIGH', 'icu_los_day_LOW', 'icu_los_day_NORMAL', 'icu_los_day_HIGH', 'iv_day_1_LOW', 'iv_day_1_NORMAL', 'iv_day_1_HIGH', 'map_1st_LOW', 'map_1st_NORMAL', 'map_1st_HIGH', 'pco2_first_LOW', 'pco2_f

In [3]:
seed_everything(42)

base_dir = f'./results/mimic-ii/explainer'
os.makedirs(base_dir, exist_ok=True)

n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
results_list = []
for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.argmax(dim=1).cpu().detach().numpy())):
    print(f'Split [{split + 1}/{n_splits}]')
    x_trainval, x_test = torch.FloatTensor(x[trainval_index]), torch.FloatTensor(x[test_index])
    y_trainval, y_test = torch.FloatTensor(y[trainval_index]), torch.FloatTensor(y[test_index])
    x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.2, random_state=42)
    print(f'{len(y_train)}/{len(y_val)}/{len(y_test)}')

    train_data = TensorDataset(x_train, y_train)
    val_data = TensorDataset(x_val, y_val)
    test_data = TensorDataset(x_test, y_test)
    train_loader = DataLoader(train_data, batch_size=train_size)
    val_loader = DataLoader(val_data, batch_size=val_size)
    test_loader = DataLoader(test_data, batch_size=test_size)

    checkpoint_callback = ModelCheckpoint(dirpath=base_dir, monitor='val_loss', save_top_k=1)
    trainer = Trainer(min_epochs=200, max_epochs=200, gpus=1, auto_lr_find=True, deterministic=True,
                      check_val_every_n_epoch=1, default_root_dir=base_dir,
                      weights_save_path=base_dir, callbacks=[checkpoint_callback])
    model = MuExplainer(n_concepts=n_concepts, n_classes=n_classes, l1=0.001, lr=0.001, explainer_hidden=[10])

    trainer.fit(model, train_loader, val_loader)
    model.freeze()
    model_results = trainer.test(model, test_dataloaders=test_loader)
    results = model.explain_class(val_loader, test_loader, target_class=1,
                                  topk_explanations=1, #max_minterm_complexity=5,
                                  concept_names=concept_names)
    results['model_accuracy'] = model_results[0]['test_acc']
    print(f"Explanation: {results['explanation']}")

    results_list.append(results)

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Split [1/10]
1278/320/178



  | Name  | Type              | Params
--------------------------------------------
0 | loss  | BCEWithLogitsLoss | 0     
1 | model | Sequential        | 1.9 K 
--------------------------------------------
1.9 K     Trainable params
0         Non-trainable params
1.9 K     Total params
0.008     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.909604549407959, 'test_acc_epoch': 0.909604549407959}
--------------------------------------------------------------------------------
Explanation: afib_flg & stroke_flg & age_HIGH & ~age_LOW & ~sapsi_first_LOW


In [4]:
results_df = pd.DataFrame(results_list)
print(results_df)
print(list(results_df['explanation'].values))
print(results_df.mean())

   target_class                                        explanation  \
0             1  afib_flg & stroke_flg & age_HIGH & ~age_LOW & ...   
1             1  stroke_flg & age_HIGH & ~resp_flg & ~age_LOW &...   
2             1  afib_flg & stroke_flg & age_HIGH & ~age_LOW & ...   
3             1  stroke_flg & age_HIGH & ~age_LOW & ~sapsi_firs...   
4             1  stroke_flg & age_HIGH & ~age_LOW & ~sapsi_firs...   
5             1  stroke_flg & age_HIGH & hr_1st_LOW & ~age_LOW ...   
6             1  afib_flg & stroke_flg & age_HIGH & ~age_LOW & ...   
7             1  stroke_flg & age_HIGH & ~age_LOW & ~sapsi_firs...   
8             1  stroke_flg & age_HIGH & ~age_LOW & ~hgb_first_...   
9             1  afib_flg & stroke_flg & age_HIGH & ~age_LOW & ...   

   explanation_accuracy  explanation_fidelity  explanation_complexity  \
0              0.859551              0.960674                       6   
1              0.870787              0.966292                       7   
2         

In [11]:
dt_scores, rf_scores = [], []
for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.argmax(dim=1).cpu().detach().numpy())):
    print(f'Split [{split + 1}/{n_splits}]')
    x_trainval, x_test = x[trainval_index], x[test_index]
    y_trainval, y_test = y[trainval_index].argmax(dim=1), y[test_index].argmax(dim=1)

    dt_model = DecisionTreeClassifier(max_depth=9, random_state=split)
    dt_model.fit(x_trainval, y_trainval)
    dt_scores.append(dt_model.score(x_test, y_test))

    rf_model = RandomForestClassifier(random_state=split)
    rf_model.fit(x_trainval, y_trainval)
    rf_scores.append(rf_model.score(x_test, y_test))

print(f'Random forest scores: {np.mean(rf_scores)} (+/- {np.std(rf_scores)})')
print(f'Decision tree scores: {np.mean(dt_scores)} (+/- {np.std(dt_scores)})')
print(f'Mu net scores (model): {results_df["model_accuracy"].mean()} (+/- {results_df["model_accuracy"].std()})')
print(f'Mu net scores (exp): {results_df["explanation_accuracy"].mean()} (+/- {results_df["explanation_accuracy"].std()})')

Split [1/10]
Split [2/10]
Split [3/10]
Split [4/10]
Split [5/10]
Split [6/10]
Split [7/10]
Split [8/10]
Split [9/10]
Split [10/10]
Random forest scores: 0.8727512219894624 (+/- 0.012337879659794616)
Decision tree scores: 0.8581095664317908 (+/- 0.019610884733861602)
Mu net scores (model): 0.8783946096897125 (+/- 0.021338360558845217)
Mu net scores (exp): 0.8721894242366532 (+/- 0.02086922680479764)
